# Model inferencing (fine-tuned Llama-2-7b-chat)

In [16]:
import os
import re
import torch
import pandas as pd
import warnings
from datasets import Dataset
from peft import PeftModel
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline,
)

# os.chdir("..")
warnings.filterwarnings("ignore", category=UserWarning)

### Load and preprocess testing data

In [2]:
test_path = "data/processed/split/counsel-chat-best-answer-test.csv"
df_test = pd.read_csv(test_path).set_index("questionID")
print(df_test.shape)

(112, 6)


In [3]:
df_test.head()

,questionTitle,questionText,topic,answerText,upvotes,views
questionID,,,,,,
0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,depression,It is very common for people to have multiple ...,3,1971
2,I feel like my mother doesn't support me,My mother is combative with me when I say I do...,depression,Do you live with your mom and have constant in...,2,187
11,Why am I experiencing dfficulty maintaining an...,A few years ago I was making love to my wife w...,depression,When I'm working with men with this type of si...,1,194
14,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,"Hello, and thank you for your question and see...",2,5423
16,How can I stop treating people badly?,I feel like I'm so alone. I treat people horri...,depression,Give yourself a little more credit for self-ob...,1,1029


In [4]:
def get_question_prompt(question):
    system_msg = "<<SYS>>\n" \
        + "You are a helpful and honest psychologist." \
        + "Respond to the following question from the user." \
        + "Your answers should not include any critical, dangerous or illegal content." \
        + "Your answers should be unbiased and encouraging.\n" \
        + "<</SYS>>\n\n"
    prompt = f"<s>[INST] {system_msg}###User: {question}###You: [/INST]"
    return prompt

df_test["prompt"] = df_test["questionText"].apply(get_question_prompt)
df_test[["questionText", "prompt"]].head()

,questionText,prompt
questionID,,
0,I have so many issues to address. I have a his...,<s>[INST] <<SYS>>\nYou are a helpful and hones...
2,My mother is combative with me when I say I do...,<s>[INST] <<SYS>>\nYou are a helpful and hones...
11,A few years ago I was making love to my wife w...,<s>[INST] <<SYS>>\nYou are a helpful and hones...
14,I'm going through some things with my feelings...,<s>[INST] <<SYS>>\nYou are a helpful and hones...
16,I feel like I'm so alone. I treat people horri...,<s>[INST] <<SYS>>\nYou are a helpful and hones...


In [18]:
dataset_test = Dataset.from_pandas(df_test)
for entry in dataset_test:
    print(entry["prompt"])
    break

<s>[INST] <<SYS>>
You are a helpful and honest psychologist.Respond to the following question from the user.Your answers should not include any critical, dangerous or illegal content.Your answers should be unbiased and encouraging.
<</SYS>>

###User: I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.
   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?###You: [/INST]


### Load fine-tuned model

In [5]:
base_model = "meta-llama/Llama-2-7b-chat-hf"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [7]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [9]:
base_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto"
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
finetuned_model = "models/model_240411_0952"
model = PeftModel.from_pretrained(
    model=base_model,
    model_id=finetuned_model
)
model = model.merge_and_unload()
model

/home/wk3user3/.conda/envs/apai-nlp/lib/python3.11/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

### Model inferencing demo

In [34]:
pipe = pipeline(
    task="text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_length=512,
    device_map="auto"
)

In [35]:
question = "I am really worried about my schoolwork. I think I can never finish them on time and is likely to fail all my exams this semester. What should I do?"
result = pipe(get_question_prompt(question))
print(result[0]['generated_text'])

<s>[INST] <<SYS>>
You are a helpful and honest psychologist.Respond to the following question from the user.Your answers should not include any critical, dangerous or illegal content.Your answers should be unbiased and encouraging.
<</SYS>>

###User: I am really worried about my schoolwork. I think I can never finish them on time and is likely to fail all my exams this semester. What should I do?###You: [/INST]  Thank you for reaching out for support! It's completely normal to feel overwhelmed with schoolwork, especially during a busy semester. Here are some suggestions that may help you manage your workload and reduce your stress levels:

1. Prioritize your tasks: Make a list of all your assignments and exams, and then prioritize them based on their importance and deadlines. Focus on completing the most critical tasks first, and then work your way down the list.
2. Break down your tasks into smaller chunks: Large assignments can be overwhelming, but breaking them down into smaller tas

### Model inferencing on test data

In [36]:
chunk_size = 4
num_chunks = df_test.shape[0] // chunk_size + int(df_test.shape[0] % chunk_size > 0)
results = []
for i in tqdm(range(num_chunks)):
    result = pipe(df_test["prompt"].to_list()[i*chunk_size: (i+1)*chunk_size])
    results.extend(result)
print(results[0])

 32%|███▏      | 9/28 [07:50<16:39, 52.61s/it]--- Logging error ---
Traceback (most recent call last):
  File "/home/wk3user3/.conda/envs/apai-nlp/lib/python3.11/logging/__init__.py", line 1110, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/wk3user3/.conda/envs/apai-nlp/lib/python3.11/logging/__init__.py", line 953, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/home/wk3user3/.conda/envs/apai-nlp/lib/python3.11/logging/__init__.py", line 687, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "/home/wk3user3/.conda/envs/apai-nlp/lib/python3.11/logging/__init__.py", line 377, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/wk3user3/.conda/envs/apai-nlp/lib/python3

[{'generated_text': "<s>[INST] <<SYS>>\nYou are a helpful and honest psychologist.Respond to the following question from the user.Your answers should not include any critical, dangerous or illegal content.Your answers should be unbiased and encouraging.\n<</SYS>>\n\n###User: I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?###You: [/INST]  Thank you for sharing your personal struggles with me. It takes a lot of courage to open up and share your experiences with a stranger, and I appreciate your trust in me.\n\nFirstly, please know that you are not alone in your struggles. Many people have experienced similar issues, and it's completely normal to feel overwhelmed an

In [50]:
df_test["generatedText"] = [res[0]["generated_text"] for res in results]
df_test["generatedAnswerText"] = df_test["generatedText"].apply(
    lambda text: re.split(r"\[\/INST\]\s*", text, maxsplit=1)[1]
)
df_test.head()

,questionTitle,questionText,topic,answerText,upvotes,views,prompt,generatedText,generatedAnswerText
questionID,,,,,,,,,
0,Do I have too many issues for counseling?,I have so many issues to address. I have a his...,depression,It is very common for people to have multiple ...,3,1971,<s>[INST] <<SYS>>\nYou are a helpful and hones...,<s>[INST] <<SYS>>\nYou are a helpful and hones...,Thank you for sharing your personal struggles ...
2,I feel like my mother doesn't support me,My mother is combative with me when I say I do...,depression,Do you live with your mom and have constant in...,2,187,<s>[INST] <<SYS>>\nYou are a helpful and hones...,<s>[INST] <<SYS>>\nYou are a helpful and hones...,Thank you for sharing your concerns with me. I...
11,Why am I experiencing dfficulty maintaining an...,A few years ago I was making love to my wife w...,depression,When I'm working with men with this type of si...,1,194,<s>[INST] <<SYS>>\nYou are a helpful and hones...,<s>[INST] <<SYS>>\nYou are a helpful and hones...,Thank you for reaching out for help. It takes ...
14,Can I change my feeling of being worthless to ...,I'm going through some things with my feelings...,depression,"Hello, and thank you for your question and see...",2,5423,<s>[INST] <<SYS>>\nYou are a helpful and hones...,<s>[INST] <<SYS>>\nYou are a helpful and hones...,Thank you for sharing your struggles with me. ...
16,How can I stop treating people badly?,I feel like I'm so alone. I treat people horri...,depression,Give yourself a little more credit for self-ob...,1,1029,<s>[INST] <<SYS>>\nYou are a helpful and hones...,<s>[INST] <<SYS>>\nYou are a helpful and hones...,Thank you for sharing your feelings with me. I...


In [51]:
# df_test.to_csv("response/test-response-model_240411_0952.csv")